In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import spacy
from torchtext.legacy.data import Field, Iterator, TabularDataset
import math

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.cuda.get_device_name(0)

cuda


'GeForce GTX 1050 Ti'

In [3]:
spacy = spacy.load('en_core_web_sm')
def tokenize(text):
    return [tok.text for tok in spacy.tokenizer(text)]

In [4]:
question = Field(sequential=True, use_vocab=True, tokenize=tokenize)
score = Field(sequential=False, use_vocab=False)
fields = {'question1': ('q1', question), 'question2': ('q2', question), 'is_duplicate':  ('s', score)}

In [5]:
train_data, test_data = TabularDataset.splits(
    path='',
    train='train.json',
    test='test.json',
    format='json',
    fields=fields
)

In [6]:
question.build_vocab(train_data)

In [7]:
train_iterator, test_iterator = Iterator.splits(
    (train_data, test_data),
    batch_size=32,
    device=device,
)

In [8]:
class SiameseLSTM(nn.Module):
    def __init__(self, input_dim, embedding_size, hidden_size, num_layers, dropout):
        super(SiameseLSTM, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(input_dim, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout)
        self.sigmoid = nn.Sigmoid()
        
    def forward_one(self, x):
        embedding = self.dropout(self.embedding(x))
        x, _ = self.rnn(embedding)
        return x
        
    def calc_dist(self, x1, x2):
        return 0
    
    def forward(self, x1, x2):
        x1 = self.forward_one(x1)
        x2 = self.forward_one(x2)
        out = self.calc_dist(x1, x2)
        return out

In [9]:
# Training Hyper parameters
num_epochs = 20
learning_rate = 0.001
batch_size = 32

In [10]:
# Model Hyper parameters
load_model = False
inpt_size = len(question.vocab)
hidden = 1024
layers = 2
dropout = 0.5
embed = 300

In [11]:
model = SiameseLSTM(inpt_size, embed, hidden, layers, dropout).to(device)

In [12]:
err = nn.CrossEntropyLoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
for epoch in range(num_epochs):
    count = 0
    print(f'epoch {epoch+1} of {num_epochs}')
    for _, batch in enumerate(train_iterator):
        q1, q2, score = batch.q1.to(device), batch.q2.to(device), batch.s.to(device)
        output = model.forward(q1, q2)
        output = output.squeeze()
        output = torch.einsum("ij->ji",output)
        print(output.shape)
        break
        optim.zero_grad()
        loss = err(output, score)
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
        optim.step()
        count += 1
        if count % 50 == 0:
            print(f'Loss is {loss}')
    break

epoch 1 of 20


AttributeError: 'int' object has no attribute 'squeeze'